In [ ]:
# prompt: provide a base template code for processing audio data to pytorch tensor

import torch
import librosa
import numpy as np

def audio_to_tensor(audio_path, sample_rate=16000):
  """
  Converts an audio file to a PyTorch tensor.

  Args:
    audio_path (str): Path to the audio file.
    sample_rate (int): Sample rate of the audio file.

  Returns:
    torch.Tensor: The audio data as a PyTorch tensor.
  """

  # Load the audio file
  audio, sr = librosa.load(audio_path, sr=sample_rate)

  # Convert the audio to a NumPy array
  audio = np.array(audio, dtype=np.float32)

  # Create a PyTorch tensor from the audio array
  tensor = torch.from_numpy(audio)

  return tensor


In [ ]:
import os
import torch
import torch.nn.functional as F

M = 3872848
data = []
music_files = os.listdir("./")[1:]
m = 0
for music_file in music_files:
  try:
    audio_data = audio_to_tensor(music_file)
    audio_data = F.pad(audio_data.reshape(1,-1), (0,M-audio_data.shape[0],0,0), mode="constant", value=0)
    data.append(audio_data)
    print(audio_data.shape)
    m = max(m, audio_data.shape[0])
  except UserWarning as e:
    pass

  except IsADirectoryError as e:
    pass
print(m)
data = torch.stack(data)
print(data.shape)

torch.Size([1, 3872848])


<ipython-input-1-77ecda3be49b>:20: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(audio_path, sr=sample_rate)


torch.Size([1, 3872848])
torch.Size([1, 3872848])
torch.Size([1, 3872848])
torch.Size([1, 3872848])
torch.Size([1, 3872848])


<ipython-input-1-77ecda3be49b>:20: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(audio_path, sr=sample_rate)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


torch.Size([1, 3872848])


<ipython-input-1-77ecda3be49b>:20: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(audio_path, sr=sample_rate)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


torch.Size([1, 3872848])
torch.Size([1, 3872848])
1
torch.Size([9, 1, 3872848])


<ipython-input-1-77ecda3be49b>:20: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(audio_path, sr=sample_rate)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [ ]:
import torch
import torch.nn.functional as F

# Example tensor
tensor = torch.tensor([[1, 2], [3, 4]])

# Pad the tensor with zeros to the desired shape
padded_tensor = F.pad(tensor, (0, 1, 0, 0), mode='constant', value=0)

print("Original tensor:")
print(tensor)
print("Padded tensor:")
print(padded_tensor)


Original tensor:
tensor([[1, 2],
        [3, 4]])
Padded tensor:
tensor([[1, 2, 0],
        [3, 4, 0]])


In [ ]:
audio_path = "Dippam_Dappam__From__Kaathuvaakula_Rendu_Kaadhal__(256k).mp3"
audio_data = audio_to_tensor(audio_path)
print(audio_data)

tensor([ 1.1842e-07,  1.0299e-07,  1.6614e-08,  ..., -1.0779e-06,
         4.3498e-05, -4.0521e-05])


In [ ]:
# prompt: now make a function to convert pytorch tensor back to wav or mp3 and save , things to consider AttributeError: No librosa attribute output

import torch
import librosa
import numpy as np
from scipy.io import wavfile


def tensor_to_audio(tensor, sample_rate=16000, output_path="audio.wav"):
  """
  Converts a PyTorch tensor to an audio file.

  Args:
    tensor (torch.Tensor): The audio data as a PyTorch tensor.
    sample_rate (int): Sample rate of the audio file.
    output_path (str): Path to save the audio file.

  Returns:
    None
  """

  # Convert the tensor to a NumPy array
  audio = tensor.numpy()

  # Convert the audio array to a WAV file
  print('aga')
  wavfile.write('Reconstructedsongforcontin'+'.wav', sample_rate, audio)
  # wavfile.write('origal_songforcontin.wav', sample_rate, original_song)
  # pickle.dump(enc_song, open( "enc_song.p", "wb" ) )

# audio_data = torch.randn(10000)
# tensor_to_audio(audio_data)


In [ ]:
import torch.nn as nn

class AutoEncoder(nn.Module):

  def __init__(self):
    super().__init__()

    self.encoder = nn.Sequential(
        nn.Linear(M,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,12),
        nn.ReLU(),
        nn.Linear(12,6)  # N * 3

    )


    self.decoder = nn.Sequential(
        nn.Linear(6,12),
        nn.ReLU(),
        nn.Linear(12,64),
        nn.ReLU(),
        nn.Linear(64,128),
        nn.ReLU(),
        nn.Linear(128,M),  # N * 3
    )


  def forward(self,x):

    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    # Note if [-1,1] use Tanh
    return decoded


In [ ]:
model = AutoEncoder().cuda()

criterion = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(),lr = 1e-2)


In [ ]:
train_data = data[0:2].reshape(2,-1).cuda()
print(train_data.shape)

torch.Size([2, 3872848])


In [ ]:
num_epochs = 10
outputs = []

for epoch in range(num_epochs):
  # for (audio_data,_) in dataloader:
  recon = model(train_data)
  loss = criterion(recon,train_data)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  torch.empty_cache()

  print(f'Epoch:{epoch}, Loss:{loss.item():.4f}')
  # outputs.append((epoch,audio_data,recon))



OutOfMemoryError: CUDA out of memory. Tried to allocate 1.85 GiB. GPU 0 has a total capacty of 14.75 GiB of which 1.47 GiB is free. Process 5678 has 13.28 GiB memory in use. Of the allocated memory 13.14 GiB is allocated by PyTorch, and 6.68 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
output = model(audio_data)

In [ ]:
output

tensor([[-0.0444, -0.0194,  0.0690,  ..., -0.0640,  0.0287,  0.0246]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
audio_data

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -3.0052e-05,
         -8.0639e-05, -7.4698e-05]], device='cuda:0')

In [ ]:
output = output.cpu().detach()

In [ ]:
output = output.reshape(-1)
print(output.shape)

torch.Size([242304])


In [ ]:
print(output.min(), output.max())

tensor(-0.4705) tensor(0.4394)


In [ ]:
tensor_to_audio(output)

aga


In [ ]:
# output = output.clamp(min=0, max=32767)
!pip install noisereduce --q

In [ ]:
from scipy.io import wavfile
import noisereduce as nr
# load data
rate, data = wavfile.read("Reconstructedsongforcontin.wav")
# perform noise reduction
print(rate)
reduced_noise = nr.reduce_noise(y=data, sr=rate)
wavfile.write("mywav_reduced_noise.wav", rate, reduced_noise)

16000
